In [2]:
should_print = True
import random
from quBLP.problemtemplate import FacilityLocationProblem as FLP
from quBLP.problemtemplate import GraphColoringProblem as GCP
from quBLP.problemtemplate import KPartitionProblem as KPP
from quBLP.models import CircuitOption, OptimizerOption
from quBLP.models import ConstrainedBinaryOptimization
from quBLP.analysis import generater

random.seed(0x7f2f)
optimizer_option = OptimizerOption(
    params_optimization_method='COBYLA',
    max_iter=150,
    opt_id=None,
)
circuit_option = CircuitOption(
    num_layers=1,
    need_draw=False,
    use_decompose=True,
    mcx_mode='constant',
    circuit_type='qiskit',
    # backend='ibm_osaka',  # 'FakeQuebec' # 'AerSimulator'
    backend='AerSimulator',  # FakeKyiv, FakeTorino, FakeBrisbane
    # feedback=['depth', 'culled_depth', 'latency', 'width'],
    # feedback=['depth', 'culled_depth', 'transpile_time', 'rss_usage'],
    # IBM=True,
)
methods = ['penalty', 'cyclic', 'commute', 'HEA']
raw_depth = [[] for _ in range(len(methods))]
depth_without_one_qubit_gate = [[] for _ in range(len(methods))]
# latency = [[] for _ in range(len(methods))]
flp_problems, flp_configs = generater.generate_flp(0, [(1, 2), (2, 3), (3, 3), (3, 4)], 1, 20)
gcp_problems, gcp_configs = generater.generate_gcp(1, [(3, 1), (3, 2), (4, 2), (4, 3)])
# kpp_problems, kpp_configs = generater.generate_kpp(1, [(4, [2, 2], 3), (6, [2, 2, 2], 5), (8, [2, 2, 4], 7), (9, [3, 3, 3], 8)], 1, 20)
# kpp_problems, kpp_configs = generater.generate_kpp(1, [(4, [2, 2], 3), (6, [2, 2, 2], 5), (8, [2, 2, 4], 7), (9, [3, 3, 3], 8)], 1, 20)
kpp_problems, kpp_configs = generater.generate_kpp(0, [(4, 2, 3), (6, 3, 5), (8, 3, 7), (9, 3, 8)], 1, 20)

problems_pkg = flp_problems + gcp_problems + kpp_problems
problems = [prb for problems in problems_pkg for prb in problems]

# all_configs = [flp_configs, gcp_configs, kpp_configs]
# problem_types = ["FLP", "GCP", "KPP"]

# for problem_type, configs in zip(problem_types, all_configs):
#     print(f"{problem_type} Configurations:")
#     for config in configs:
#         print(*config)
#     print()
prb : ConstrainedBinaryOptimization = problems[1]
prb.set_algorithm_optimization_method('commute', 400)
# print(prb.get_best_cost())
# print(flp.get_solution_bitstr())
# exit()
print(prb.optimize(optimizer_option, circuit_option))
# print(prb.dichotomy_optimize(optimizer_option, circuit_option, num_frozen_qubit=1))

# print(kpp_configs)

ImportError: libcublas.so.11: cannot open shared object file: No such file or directory

In [4]:
from qiskit_aer import AerSimulator
AerSimulator(method='statevector', device='GPU')

ImportError: libcublas.so.11: cannot open shared object file: No such file or directory

In [1]:
from qiskit_aer import AerSimulator
AerSimulator(method='statevector', device='GPU')

ImportError: libcublas.so.11: cannot open shared object file: No such file or directory

In [ ]:
def process_objective_term_list(objective_iterm_list, frozen_idx_list, frozen_state_list):
    process_list = []
    zero_indices = [frozen_idx_list[i] for i, x in enumerate(frozen_state_list) if x == 0]
    nonzero_indices = [i for i in frozen_idx_list if i not in zero_indices]
    frozen_idx_list = np.array(frozen_idx_list)
    print(zero_indices, nonzero_indices)
    for dimension in objective_iterm_list:
        dimension_list = []
        for objective_term in dimension:
            if any(idx in objective_term[0] for idx in zero_indices):
                # 如果 frozen_state == 0 且 x 在内层列表中，移除整个iterm
                continue
            else:
                # 如果 frozen_state == 1 且 x 在内层列表中，移除iterm中的 x
                iterm = [varbs for varbs in objective_term[0] if varbs not in nonzero_indices]
                iterm = [x - np.sum(frozen_idx_list < x) for x in iterm]
                if iterm:
                    dimension_list.append((iterm, objective_term[1]))
        # 空维度也要占位
        process_list.append(dimension_list)
    return process_list
process_objective_term_list([[([0, 1, 2], 20), ([1, 2, 3], 14), ([2, 3, 4], 19), ([3], 18)], []], [2, 3], [1, 1])

NameError: name 'np' is not defined

In [ ]:
frozen_idx = [1]
any(idx in [1,2,3] for idx in frozen_idx)

True

In [ ]:
a = 1

In [ ]:
import gurobipy as gp
import numpy as np

def find_feasible_solution_with_gurobi(A, fixed_values=None):
    num_vars = A.shape[1] - 1  # Number of variables
    # Create a new model
    model = gp.Model("feasible_solution")
    model.setParam('OutputFlag', 0)
    # Create variables
    variables = []
    for i in range(num_vars):
        variables.append(model.addVar(vtype=gp.GRB.BINARY, name=f"x{i}"))
    # Set objective (minimization problem, but no objective here since we just need a feasible solution)
    # Add constraints
    for i in range(A.shape[0]):
        lhs = gp.quicksum(A[i, j] * variables[j] for j in range(num_vars))
        rhs = A[i, -1]
        model.addConstr(lhs == rhs)
        
    # Add fixed values constraints
    if fixed_values:
        for i in range(num_vars):
            if fixed_values[i] is not None:
                model.addConstr(variables[i] == fixed_values[i])
    
    model.optimize()
    
    if model.status == gp.GRB.Status.OPTIMAL:
        # Retrieve solution
        solution = [int(variables[i].x) for i in range(num_vars)]
        return solution
    else:
        print("No feasible solution found.")
        return None

# Example usage:
A = np.array([
    [-1,  0,  1,  0,  1,  0,  0],
    [ 0, -1,  0,  1,  0,  1,  0],
    [ 0,  0,  1,  1,  0,  0,  1]
])

fixed_values = [1, 0, None, None, None, None]  # Example fixed values

solution = find_feasible_solution_with_gurobi(A, fixed_values)
if solution is not None:
    print("Feasible solution found:", solution)


Restricted license - for non-production use only - expires 2025-11-24
Feasible solution found: [1, 0, 1, 0, 0, 0]


In [ ]:
def partition(number, k):
    answer = []
    
    def helper(n, k, start, current):
        if k == 1:
            if n >= start:
                answer.append(current + [n])
            return
        for i in range(start, n - k + 2):
            helper(n - i, k - 1, i, current + [i])
    
    helper(number, k, 1, [])
    return answer

# 示例
print(partition(9, 3))  # [[1, 3], [2, 2]]


[[1, 1, 7], [1, 2, 6], [1, 3, 5], [1, 4, 4], [2, 2, 5], [2, 3, 4], [3, 3, 3]]


In [ ]:
def process_objective_term_list(objective_iterm_list, frozen_idx, frozen_state):
    process_list = []
    for dimension in objective_iterm_list:
        dimension_list = []
        for objective_term in dimension:
            if frozen_state == 0 and frozen_idx in objective_term[0]:
                # 如果 frozen_state == 0 且 x 在内层列表中，移除整个term
                continue
            elif frozen_state == 1 and frozen_idx in objective_term[0]:
                # 如果 frozen_state == 1 且 x 在内层列表中，移除term中的 x
                iterm = [varbs for varbs in objective_term[0] if varbs != frozen_idx]
                if iterm:
                    dimension_list.append([iterm, objective_term[1]])
            else:
                # 否则保留 inner_list
                dimension_list.append(objective_term)
        # 空维度也要占位
        process_list.append(dimension_list)
    return process_list

# 示例使用
nested_list = [[[[0], 14], [[1, 0], 11], [[2], 9], [[3], 5]], [[[0, 1], 1], [[1, 2], 1]],[[[2], 1]],[[[2,3], 1]]]

new_list = process_objective_term_list(nested_list, 2, 1)
print(new_list)


[[[[0], 14], [[1, 0], 11], [[3], 5]], [[[0, 1], 1], [[1], 1]], [], [[[3], 1]]]


In [ ]:
import numpy as np

# 给定的矩阵
matrix = np.array([
    [-1., 1., -1., 1., 0., 0.],
    [1., 0., 0., 0., 1., 0.],
    [0., 1., 0., 0., 0., 1.]
])

# 找到具有最多非零元素的列索引
non_zero_counts = np.count_nonzero(matrix, axis=0)
max_non_zero_col_index = np.argmax(non_zero_counts)

# 删除该列
matrix_new = np.delete(matrix, max_non_zero_col_index, axis=1)

matrix_new


array([[1., 0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0., 1.]])

In [ ]:
import pennylane as qml
dev = qml.device("default.qubit", wires=1)
@qml.qnode(dev, diff_method="adjoint")
def circuit():
    qml.PauliX(0)
    return qml.expval(qml.PauliZ(0))
print(circuit())

-1.0


In [ ]:
import gurobipy as gp
from gurobipy import gp.GRB

# 创建一个新的优化模型
model = gp.Model("product_maximization")

# 定义变量
x1 = model.addVar(vtype=gp.GRB.CONTINUOUS, name="x1")
x2 = model.addVar(vtype=gp.GRB.CONTINUOUS, name="x2")
x3 = model.addVar(vtype=gp.GRB.CONTINUOUS, name="x3")
z = model.addVar(vtype=gp.GRB.CONTINUOUS, name="z")



# 设置优化目标：最大化 z
model.setObjective(x1 * x3 * 2, gp.GRB.MAXIMIZE)

# 求解优化问题
model.optimize()

# 输出结果
if model.status == gp.GRB.OPTIMAL:
    print(f"Optimal solution found: x1 = {x1.x}, x2 = {x2.x}, x3 = {x3.x}, z = {z.x}")
    print(f"Product x1 * x2 * x3 = {x1.x * x2.x * x3.x}")
else:
    print("No optimal solution found.")

# 释放资源
model.dispose()


Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: AMD EPYC 9554 64-Core Processor, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 127 physical cores, 255 logical processors, using up to 32 threads

Optimize a model with 0 rows, 4 columns and 0 nonzeros
Model fingerprint: 0xccc53ee8
Model has 1 quadratic objective term
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [4e+00, 4e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [0e+00, 0e+00]
Presolve removed 0 rows and 2 columns

Continuous model is non-convex -- solving as a MIP

Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 8.000000e+18
Presolve removed 0 rows and 2 columns
Presolve time: 0.00s
Presolved: 2 rows, 4 columns, 4 nonzeros
Presolved model has 1 bilinear constraint(s)
         in product terms.
         Presolve was not able to compute smaller bounds for these variables.

In [ ]:
def generate_terms(x, n, m):
    from itertools import combinations
    total_prod = 0
    terms = []

    for j in range(n):
        term_sum = 1
        current_terms = []
        for k in range(1, m + 1):
            for combo in combinations(range(m), k):
                term = (-1) ** k
                term_str = ""
                for index in combo:
                    term *= x[index][j]
                    term_str += f"x{index}{j} "
                term_sum += term
                current_terms.append((term, term_str.strip()))
        print(':', term_sum)
        total_prod += - term_sum
        print(total_prod)
        terms.append(current_terms)
    return total_prod, terms

# Example usage:
frozen_idx = [[0, 0], [0, 1], [0, 0]]  # Example values for x_ij
n = 2  # Example number of sums
m = 3  # Example number of products

total, terms = generate_terms(frozen_idx, n, m)

print("没用上颜色 Total sum:", total)
print("\nAll terms:")
for term_set in terms:
    for iterm, term_str in term_set:
        print(f"{iterm}: {term_str}")
    print()


: 1
-1
: 0
-1
没用上颜色 Total sum: -1

All terms:
0: x00
0: x10
0: x20
0: x00 x10
0: x00 x20
0: x10 x20
0: x00 x10 x20

0: x01
-1: x11
0: x21
0: x01 x11
0: x01 x21
0: x11 x21
0: x01 x11 x21



In [ ]:
from typing import List, Tuple

# 定义类型别名，用于描述每个次项
# 定义 objective_func_term_list 的类型注解
objective_func_term_list: List[List[Tuple[List[int], float]]]

# 示例数据
objective_func_term_list = [
    [[[0], 2.5], [[1], 3.0]],    # 一次项列表
    [[[0, 1], 1.5]],             # 二次项列表
    [[[0, 1, 2], 2.0]]           # 三次项列表
]

# 演示如何使用类型注解
for term_list in objective_func_term_list:
    for iterm in term_list:
        variable_indices, coefficient = iterm
        # 现在 variable_indices 是 List[int] 类型，coefficient 是 float 类型
        print(f"变量序号: {variable_indices}, 系数: {coefficient}")


变量序号: [0], 系数: 2.5
变量序号: [1], 系数: 3.0
变量序号: [0, 1], 系数: 1.5
变量序号: [0, 1, 2], 系数: 2.0


In [ ]:
a = [1]
def add(a):
  a[0] += 1

add(a)
a

[2]

In [ ]:
from qiskit import QuantumCircuit, transpile
import numpy as np
from scipy.linalg import expm
from qiskit_aer import Aer
from itertools import combinations
def get_circ_unitary(quantum_circuit):
    backend = Aer.get_backend('unitary_simulator' )
    new_circuit = transpile(quantum_circuit, backend)
    job = backend.run(new_circuit)
    result = job.result()
    unitary = result.get_unitary()
    from quBLP.utils.linear_system import reorder_tensor_product
    reoder_unitary = reorder_tensor_product(np.array(unitary))
    # 张量积逆序
    return reoder_unitary
def Ho_decompose(qc:QuantumCircuit, objective_func_term_list, param):
    for n_power_term_list in objective_func_term_list:
        for var_idx_list, theta in n_power_term_list :
            pow = len(var_idx_list)
            for k in range(1, pow + 1):
                fianl_theta = (1 / 2) ** pow * 2 * theta * (-1) ** k
                for combo in combinations(range(pow), k):
                    for i in range(len(combo) - 1):
                        qc.cx(var_idx_list[combo[i]], var_idx_list[combo[i + 1]])
                    qc.rz(fianl_theta * param, var_idx_list[combo[-1]])
                    for i in range(len(combo) - 2, -1, -1):
                        qc.cx(var_idx_list[combo[i]], var_idx_list[combo[i + 1]])

    # # 一次项
    # for [var_idx], theta in objective_func_term_list[0]:
    #     qc.rz(-theta * param, var_idx)
    # # 二次项
    # for [var_idx_1, var_idx_2], theta in objective_func_term_list[1]:
    #     qc.rz(-theta * param/ 2, var_idx_1)
    #     qc.rz(-theta * param/ 2, var_idx_2)
    #     qc.cx(var_idx_1, var_idx_2)
    #     qc.rz(theta / 2 * param, var_idx_2)
    #     qc.cx(var_idx_1, var_idx_2)

t = 0.5
qc = QuantumCircuit(4)
Ho_decompose(qc, [[[[0], 1]], [[[1,2], 1.3], [[2, 0], 0.88]]], t)

c1 = get_circ_unitary(qc)
print(qc.draw())

      ┌──────────┐ ┌───────────┐                               ┌───┐»
q_0: ─┤ Rz(-0.5) ├─┤ Rz(-0.22) ├───────────────────────────────┤ X ├»
     ┌┴──────────┴┐└───────────┘                               └─┬─┘»
q_1: ┤ Rz(-0.325) ├──────■─────────────────────■─────────────────┼──»
     ├────────────┤    ┌─┴─┐    ┌───────────┐┌─┴─┐┌───────────┐  │  »
q_2: ┤ Rz(-0.325) ├────┤ X ├────┤ Rz(0.325) ├┤ X ├┤ Rz(-0.22) ├──■──»
     └────────────┘    └───┘    └───────────┘└───┘└───────────┘     »
q_3: ───────────────────────────────────────────────────────────────»
                                                                    »
«     ┌──────────┐┌───┐
«q_0: ┤ Rz(0.22) ├┤ X ├
«     └──────────┘└─┬─┘
«q_1: ──────────────┼──
«                   │  
«q_2: ──────────────■──
«                      
«q_3: ─────────────────
«                      


In [ ]:
from qiskit import QuantumCircuit, transpile
import numpy as np
from scipy.linalg import expm
from qiskit_aer import Aer
from itertools import combinations
def get_circ_unitary(quantum_circuit):
    backend = Aer.get_backend('unitary_simulator' )
    new_circuit = transpile(quantum_circuit, backend)
    job = backend.run(new_circuit)
    result = job.result()
    unitary = result.get_unitary()
    from quBLP.utils.linear_system import reorder_tensor_product
    reoder_unitary = reorder_tensor_product(np.array(unitary))
    # 张量积逆序
    return reoder_unitary
def Ho_decompose(qc:QuantumCircuit, objective_func_term_list, param):
    # 一次项
    for [var_idx], theta in objective_func_term_list[0]:
        qc.rz(-theta * param, var_idx)
    # 二次项
    for [var_idx_1, var_idx_2], theta in objective_func_term_list[1]:
        qc.rz(-theta * param/ 2, var_idx_1)
        qc.rz(-theta * param/ 2, var_idx_2)
        qc.cx(var_idx_1, var_idx_2)
        qc.rz(theta / 2 * param, var_idx_2)
        qc.cx(var_idx_1, var_idx_2)

t = 0.5
qc = QuantumCircuit(4)
Ho_decompose(qc, [[[[0], 1]], [[[1,2], 1.3], [[2, 0], 0.88]]], t)
c2 = get_circ_unitary(qc)
print(qc.draw())

      ┌──────────┐ ┌───────────┐                               ┌───┐»
q_0: ─┤ Rz(-0.5) ├─┤ Rz(-0.22) ├───────────────────────────────┤ X ├»
     ┌┴──────────┴┐└───────────┘                               └─┬─┘»
q_1: ┤ Rz(-0.325) ├──────■─────────────────────■─────────────────┼──»
     ├────────────┤    ┌─┴─┐    ┌───────────┐┌─┴─┐┌───────────┐  │  »
q_2: ┤ Rz(-0.325) ├────┤ X ├────┤ Rz(0.325) ├┤ X ├┤ Rz(-0.22) ├──■──»
     └────────────┘    └───┘    └───────────┘└───┘└───────────┘     »
q_3: ───────────────────────────────────────────────────────────────»
                                                                    »
«     ┌──────────┐┌───┐
«q_0: ┤ Rz(0.22) ├┤ X ├
«     └──────────┘└─┬─┘
«q_1: ──────────────┼──
«                   │  
«q_2: ──────────────■──
«                      
«q_3: ─────────────────
«                      


In [ ]:
np.allclose(c1, c2)

True

In [ ]:
from qiskit import QuantumCircuit, transpile
import numpy as np
from scipy.linalg import expm
from qiskit_aer import Aer
# 输入qc, 返回电路酉矩阵
def get_circ_unitary(quantum_circuit):
    backend = Aer.get_backend('unitary_simulator' )
    new_circuit = transpile(quantum_circuit, backend)
    job = backend.run(new_circuit)
    result = job.result()
    unitary = result.get_unitary()
    from quBLP.utils.linear_system import reorder_tensor_product
    reoder_unitary = reorder_tensor_product(np.array(unitary))
    # 张量积逆序
    return reoder_unitary
t = 0.5
qc = QuantumCircuit(2)
## gate for X_iX_j
qc.h(0)
qc.h(1)
qc.cx(0, 1)
qc.rz(2*t, 1)
qc.cx(0, 1)
qc.h(0)
qc.h(1)
## gate for Y_iY_j
qc.u(np.pi/2, np.pi/2, np.pi/2, 0)
qc.u(np.pi/2, np.pi/2, np.pi/2, 1)
qc.cx(1, 0)
qc.rz(2*t, 0)
qc.cx(1, 0)
qc.u(np.pi/2, np.pi/2, np.pi/2, 0)
qc.u(np.pi/2, np.pi/2, np.pi/2, 1)
print(qc.draw())
np.set_printoptions(suppress=True, precision=4,  linewidth=400)
get_circ_unitary(qc)

     ┌───┐                   ┌───┐┌────────────────┐┌───┐┌───────┐┌───┐»
q_0: ┤ H ├──■─────────────■──┤ H ├┤ U(π/2,π/2,π/2) ├┤ X ├┤ Rz(1) ├┤ X ├»
     ├───┤┌─┴─┐┌───────┐┌─┴─┐├───┤├────────────────┤└─┬─┘└───────┘└─┬─┘»
q_1: ┤ H ├┤ X ├┤ Rz(1) ├┤ X ├┤ H ├┤ U(π/2,π/2,π/2) ├──■─────────────■──»
     └───┘└───┘└───────┘└───┘└───┘└────────────────┘                   »
«     ┌────────────────┐
«q_0: ┤ U(π/2,π/2,π/2) ├
«     ├────────────────┤
«q_1: ┤ U(π/2,π/2,π/2) ├
«     └────────────────┘


array([[ 1.    -0.j    , -0.    -0.j    , -0.    -0.j    ,  0.    -0.j    ],
       [ 0.    -0.j    ,  0.5403-0.j    , -0.    -0.8415j, -0.    +0.j    ],
       [ 0.    -0.j    , -0.    -0.8415j,  0.5403-0.j    , -0.    +0.j    ],
       [-0.    +0.j    ,  0.    -0.j    , -0.    -0.j    ,  1.    -0.j    ]])

In [ ]:
gate_x = np.array([[0, 1],[1, 0]])
gate_y = np.array([[0, -1j],[1j, 0]])
expm(-1j * (np.kron(gate_x,gate_x) + np.kron(gate_y,gate_y)) * t)

array([[1.    +0.j    , 0.    +0.j    , 0.    +0.j    , 0.    +0.j    ],
       [0.    +0.j    , 0.5403+0.j    , 0.    -0.8415j, 0.    +0.j    ],
       [0.    +0.j    , 0.    -0.8415j, 0.5403+0.j    , 0.    +0.j    ],
       [0.    +0.j    , 0.    +0.j    , 0.    +0.j    , 1.    +0.j    ]])

In [ ]:
expm(-1j * np.kron(gate_x,gate_x)*t)\
  # @ expm(-1j * ( np.kron(gate_y,gate_y)) * t)


array([[0.8776+0.j    , 0.    +0.j    , 0.    +0.j    , 0.    -0.4794j],
       [0.    +0.j    , 0.8776+0.j    , 0.    -0.4794j, 0.    +0.j    ],
       [0.    +0.j    , 0.    -0.4794j, 0.8776+0.j    , 0.    +0.j    ],
       [0.    -0.4794j, 0.    +0.j    , 0.    +0.j    , 0.8776+0.j    ]])

In [ ]:
# 示例列表
list1 = ['a', 'b', 'c']
list2 = [1, 2, 3]
list3 = ['alpha', 'beta', 'gamma', 'delta', 'epsilon', 'zeta']

# 合并list1和list2
merged_list = list1 + list2

# 生成字典
result_dict = dict(zip(merged_list, list3))

print(result_dict)


{'a': 'alpha', 'b': 'beta', 'c': 'gamma', 1: 'delta', 2: 'epsilon', 3: 'zeta'}


In [ ]:
import qiskit
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister,transpile
from qiskit.circuit import QuantumCircuit, Parameter

phi = [Parameter(f'phi{i}') for i in range(4)]
phi2 = Parameter('phi2')
circuit = QuantumCircuit(4, 4)
circuit.rx(phi[0],0)
circuit.measure([0], [0])
circuit.x(0)
circuit.measure([0], [0])
circuit.measure([0], [0])
circuit.rz(2 * phi2,1)
transpile_cir = transpile(circuit, basis_gates=['ecr', 'id', 'rz','sx','x'], optimization_level=3)
# bind the parameters after circuit to create a bound circuit
bc = transpile_cir.assign_parameters({phi[0]: 0.8})
bc = transpile_cir.assign_parameters([1,2])
bc = transpile_cir.assign_parameters([1,2])
bc = transpile_cir.assign_parameters([2,4])
bc.measure(range(4), range(4)[::-1])
print(bc.draw())

global phase: 5π/4
     ┌─────────┐┌────┐┌────────────┐┌────┐┌──────────┐┌─┐┌─────────┐┌────┐»
q_0: ┤ Rz(π/2) ├┤ √X ├┤ Rz(5.1416) ├┤ √X ├┤ Rz(5π/2) ├┤M├┤ Rz(π/2) ├┤ √X ├»
     └┬───────┬┘└┬─┬─┘└────────────┘└────┘└──────────┘└╥┘└─────────┘└────┘»
q_1: ─┤ Rz(8) ├──┤M├───────────────────────────────────╫──────────────────»
      └──┬─┬──┘  └╥┘                                   ║                  »
q_2: ────┤M├──────╫────────────────────────────────────╫──────────────────»
         └╥┘      ║                                    ║                  »
q_3: ─────╫───────╫────────────────────────────────────╫──────────────────»
          ║       ║                                    ║                  »
c: 4/═════╩═══════╩════════════════════════════════════╩══════════════════»
          1       2                                    0                  »
«     ┌──────────┐┌─┐┌─┐┌─┐   
«q_0: ┤ Rz(-π/2) ├┤M├┤M├┤M├───
«     └──────────┘└╥┘└╥┘└╥┘   
«q_1: ─────────────╫──╫──╫────
«                  ║ 

In [ ]:
for i in range(10):
  a = i
print(a)

In [ ]:
import pennylane as qml
from pennylane import numpy as np

# 选择要使用的设备和量子比特数


num_qubits = 11
dev = qml.device("default.qubit", wires=num_qubits+1)
def add():
    qml.Hadamard(wires=1)
    # 在每个量子比特上应用 Hadamard 门
    for i in range(num_qubits):
        qml.Hadamard(wires=i)
    # 添加一组门（可以根据需要添加更多的门）
    qml.CNOT(wires=[0, 1])  # 0 号量子比特控制，1 号量子比特目标

from quBLP.solvers.circuits.pennylane_decompose import driver_component as driver_component_pennylane
# 定义自定义量子电路
def custom_quantum_circuit(params):
    # 创建一个新的量子电路
    @qml.qnode(dev)
    def circuit():
        # X门添加到第一个量子比特
        # qml.RZ(100, wires=0)
        qml.PauliX(wires=0)
        qml.PauliX(wires=2)
        qml.PauliX(wires=4)
        qml.RZ(1j, wires=1)
        # H门添加到第二个量子比特
        
        # add()
        return qml.probs(wires=range(num_qubits))  # 返回测量结果的概率分布
    
    # 为电路定义参数
    return circuit

# 测试函数
params = np.array([])
process_list = custom_quantum_circuit(params)()
print("Final state probabilities:", process_list)
print(f'probs_sum: {sum(process_list)}')
print(qml.draw(custom_quantum_circuit(0.1))())

In [ ]:
import pennylane as qml
from pennylane import numpy as np

# 选择要使用的设备和量子比特数


num_qubits = 11
dev = qml.device("default.qubit", wires=num_qubits+1)
def add():
    qml.Hadamard(wires=1)
    # 在每个量子比特上应用 Hadamard 门
    for i in range(num_qubits):
        qml.Hadamard(wires=i)
    # 添加一组门（可以根据需要添加更多的门）
    qml.CNOT(wires=[0, 1])  # 0 号量子比特控制，1 号量子比特目标

from quBLP.solvers.circuits.pennylane_decompose import driver_component as driver_component_pennylane
# 定义自定义量子电路
def custom_quantum_circuit(params):
    # 创建一个新的量子电路
    @qml.qnode(dev)
    def circuit():
        # X门添加到第一个量子比特
        # qml.RZ(100, wires=0)
        qml.PauliX(wires=0)
        qml.PauliX(wires=2)
        qml.PauliX(wires=4)
        driver_component_pennylane(list(range(6)),[num_qubits],[0, 1, 0, 1, 0, 1], 1j+1)
        
        # H门添加到第二个量子比特
        
        # add()
        return qml.probs(wires=range(num_qubits))  # 返回测量结果的概率分布
    
    # 为电路定义参数
    return circuit

# 测试函数
params = np.array([])
process_list = custom_quantum_circuit(params)()
print("Final state probabilities:", process_list)
print(sum(process_list))
print(qml.draw(custom_quantum_circuit(0.1))())

In [ ]:
import pennylane as qml
from pennylane import numpy as np

# 定义一个酉矩阵 U
U = np.array([
    [0, 1],
    [1, 0]
])

# 初始化量子设备
dev = qml.device('default.qubit', wires=3)

@qml.qnode(dev)
def circuit():
    qml.QuadX(0)
    qml.CNOT(wires=[0, 1])
    qml.QubitUnitary(U, wires=0)
    qml.CNOT(wires=[0, 1])
    qml.MultiControlledX(wires=[0, 1, 2])
    return qml.state()
# 执行电路
print(circuit())
print(qml.draw(circuit)())
specs = qml.specs(circuit)()
print(specs['resources'])
print(specs['resources'].depth)

In [ ]:
dev = qml.device("default.qubit", wires=[0, 1])
@qml.qnode(dev)
def circuit():
    qml.Hadamard(wires=1)
    qml.CZ(wires=[0, 1])
    qml.Hadamard(wires=1)
    return qml.state()

print(qml.matrix(circuit)().round(2))
print(qml.draw(circuit)())

In [ ]:
shots_list = [5, 10, 1000]
dev = qml.device("default.qubit", wires=2, shots=shots_list)
@qml.qnode(dev)
def circuit(x):
    qml.RX(x, wires=0)
    qml.CNOT(wires=[0, 1])
    return qml.expval(qml.PauliZ(0)), qml.expval(qml.PauliX(1)), qml.expval(qml.PauliZ(0) @ qml.PauliX(1))
circuit(0.5)

In [ ]:
def circuit1(theta):
    qml.RX(theta, wires=1)
    qml.PauliZ(0)

def circuit2(theta):
    qml.RX(theta, wires=1)
    qml.PauliZ(0)

matrix_fn = qml.matrix(circuit2)
matrix_fn(0)

In [ ]:
import numpy as np
dev_unique_wires = qml.device('default.qubit', wires=['aux', 'q1', 'q2'])
def my_quantum_function(x, y):
    qml.RZ(x, wires='q1')
    qml.CNOT(wires=['aux' ,'q1'])
    qml.RY(y, wires='q2')
    return qml.expval(qml.PauliZ('q2'))
circuit = qml.QNode(my_quantum_function, dev_unique_wires)
circuit(np.pi/4, 0.7)

In [ ]:
print(qml.draw(circuit)(np.pi/4, 0.7))

In [ ]:
import matplotlib.pyplot as plt
qml.drawer.use_style("black_white")
fig, ax = qml.draw_mpl(circuit)(np.pi/4, 0.7)
plt.show()

In [ ]:
dev = qml.device('default.qubit', wires=2)

@qml.qnode(dev)
def circuit(x):
    qml.RZ(x, wires=0)
    qml.CNOT(wires=[0,1])
    qml.RY(x, wires=1)
    return qml.expval(qml.PauliZ(1))

process_list = circuit(0.543)

In [ ]:
qml.simplify(qml.RX(4*np.pi+0.1, 0 ))

In [ ]:
qml.simplify(qml.adjoint(qml.RX(1.23, 0)))

In [ ]:
qml.simplify(qml.ops.Pow(qml.RX(1, 0), 2.1))

In [ ]:
sum_op = qml.RX(1, 0) + qml.X(0)
prod1 = qml.X(0) @ sum_op
nested_op = qml.prod(prod1, qml.RX(1, 0))
qml.simplify(nested_op)

In [ ]:
def circuit(weights):
    qml.BasicEntanglerLayers(weights, wires=[0, 1, 2])
    return qml.expval(qml.PauliZ(0))

original_dev = qml.device("default.qubit", wires=3)
original_qnode = qml.QNode(circuit, original_dev)
weights = np.array([[0.4, 0.5, 0.6]])
print(qml.draw(original_qnode, expansion_strategy="device")(weights))

In [ ]:
def custom_cnot(wires, **_):
    return [
        qml.Hadamard(wires=wires[1]),
        qml.CZ(wires=[wires[0], wires[1]]),
        qml.Hadamard(wires=wires[1])
    ]

custom_decomps = {qml.CNOT: custom_cnot}

decomp_dev = qml.device("default.qubit", wires=3, custom_decomps=custom_decomps)
decomp_qnode = qml.QNode(circuit, decomp_dev)
print(qml.draw(decomp_qnode, expansion_strategy="device")(weights))

In [ ]:
import pennylane as qml
import numpy as np

# 定义任意酉矩阵
U = np.array([[0, 1], [1, 0]])  # 这是一个非常简单的酉矩阵（Pauli-X）

# 定义一个电路
def circuit():
    qml.QubitUnitary(U, wires=0)
    return qml.expval(qml.PauliZ(0))

# 自定义分解函数
def custom_unitary_decomp(U, wires):
    # 在这里将U矩阵分解为基础门操作
    # 例如，如果U是一个简单的单量子比特门，直接使用qml.Rot来表示
    theta, phi, omega = qml.transforms.rotation_decomposition(U)
    return [qml.Rot(theta, phi, omega, wires=wires)]

# 自定义分解规则
custom_decomps = {qml.QubitUnitary: custom_unitary_decomp}

# 创建自定义设备
decomp_dev = qml.device("default.qubit", wires=1, custom_decomps=custom_decomps)

# 创建QNode
decomp_qnode = qml.QNode(circuit, decomp_dev)

# 绘制电路
print(qml.draw(decomp_qnode)())


In [ ]:
import pennylane as qml
import numpy as np

# 定义任意酉矩阵
U = np.array([[0, 1], [1, 0]])  # 这是一个简单的酉矩阵（Pauli-X）

# 定义一个电路
def circuit():
    qml.QubitUnitary(U, wires=0)
    return qml.expval(qml.PauliZ(0))

# 创建默认设备
original_dev = qml.device("default.qubit", wires=1)
original_qnode = qml.QNode(circuit, original_dev)

# 绘制原始电路
print("Original circuit:")
print(qml.draw(original_qnode)())

# 使用内置变换函数将电路中的酉矩阵分解为基础门
decomposed_circuit = qml.transforms.unitary_to_rot(circuit)

# 创建一个设备来运行分解后的电路
decomposed_qnode = qml.QNode(decomposed_circuit, original_dev)

# 绘制分解后的电路
print("Decomposed circuit:")
print(qml.draw(decomposed_qnode)())


In [ ]:
from functools import partial
import pennylane as qml
import matplotlib.pyplot as plt

dev = qml.device("default.qubit", wires=3)


def circuit(angles):
    qml.Hadamard(wires=1)
    qml.Hadamard(wires=2)
    qml.RX(angles[0], 0)
    qml.CNOT(wires=[1, 0])
    qml.CNOT(wires=[2, 1])
    qml.RX(angles[2], wires=0)
    qml.RZ(angles[1], wires=2)
    qml.CNOT(wires=[2, 1])
    qml.RZ(-angles[1], wires=2)
    qml.CNOT(wires=[1, 0])
    qml.Hadamard(wires=1)
    qml.CY(wires=[1, 2])
    qml.CNOT(wires=[1, 0])
    return qml.expval(qml.PauliZ(wires=0))


angles = [0.1, 0.3, 0.5]
qnode = qml.QNode(circuit, dev)
qml.draw_mpl(qnode, decimals=1, style="sketch")(angles)
plt.show()

In [ ]:
def custom_gates(theta):
    qml.RX(theta, wires=0)
    qml.RY(theta, wires=1)

def my_circuit(theta):
    qml.Hadamard(wires=0)
    qml.CNOT(wires=[0, 1])
    custom_gates(theta)

# 创建量子电路
dev = qml.device('default.qubit', wires=2)
@qml.qnode(dev)
def circuit_with_custom(theta):
    my_circuit(theta)
    return qml.expval(qml.PauliZ(0))

# print(qml.draw(circuit_with_custom)(0.1))
# 执行电路
theta_val = 0.1
process_list = circuit_with_custom
print(qml.draw(process_list)(theta_val))


In [ ]:
qml.Hadamard(wires=0)
@qml.qnode(qml.device('default.qubit', wires=3))
def ci():
  qml.MultiControlledX(wires=[0,2,1])
  qml.PauliX(wires=0)
  qml.PauliX(wires=1)
  return qml.state()

print(qml.draw(ci)())


In [ ]:
from time import perf_counter
from scipy.linalg import expm
import numpy as np
import pennylane as qml

def apply_convert(list_qubits, bit_string):
    num_qubits = len(bit_string)
    for i in range(0, num_qubits - 1):
        qml.CNOT(wires=[list_qubits[i + 1], list_qubits[i]])
        if bit_string[i] == bit_string[i + 1]:
            qml.PauliX(list_qubits[i])
    qml.Hadamard(list_qubits[-1])
    qml.PauliX(list_qubits[-1])

def apply_reverse(list_qubits, bit_string):
    num_qubits = len(bit_string)
    qml.PauliX(list_qubits[-1])
    qml.Hadamard(list_qubits[-1])
    for i in range(num_qubits - 2, -1, -1):
        if bit_string[i] == bit_string[i + 1]:
            qml.PauliX(list_qubits[i])
        qml.CNOT(wires=[list_qubits[i + 1], list_qubits[i]])

def decompose_phase_gate(qr, ancilla, phase):
    num_qubits = len(qr)
    if num_qubits == 1:
        qml.PhaseShift(phase, wires=qr[0])
    elif num_qubits == 2:
        qml.ControlledPhaseShift(phase, wires=qr)
    else:
        half_num_qubit = num_qubits // 2
        qr1 = qr[:half_num_qubit]
        qr2 = qr[half_num_qubit:]
        qml.RZ(-phase/2, wires=ancilla[0])
        qml.MultiControlledX(wires=list(qr1) + [ancilla[0]])
        qml.RZ(phase/2, wires=ancilla[0])
        qml.MultiControlledX(wires=list(qr2) + [ancilla[0]])
        qml.RZ(-phase/2, wires=ancilla[0])
        qml.MultiControlledX(wires=list(qr1) + [ancilla[0]])
        qml.RZ(phase/2, wires=ancilla[0])
        qml.MultiControlledX(wires=list(qr2) + [ancilla[0]])

# separate functions facilitate library calls
def driver_component(list_qubits, list_ancilla, bit_string, t):
    apply_convert(list_qubits, bit_string)
    # qml.Barrier
    decompose_phase_gate(list_qubits, list_ancilla, -t)
    # qml.Barrier
    qml.PauliX(list_qubits[-1])
    decompose_phase_gate(list_qubits, list_ancilla, t)
    qml.PauliX(list_qubits[-1])
    # qml.Barrier
    apply_reverse(list_qubits, bit_string)
    
def get_driver_component(num_qubits, t, bit_string):
    num_wires = num_qubits + 2
    dev = qml.device('default.qubit', wires=num_wires)

    @qml.qnode(dev)
    def circuit():
        driver_component(list(range(num_qubits)), [num_qubits], bit_string,  2 * np.pi * t)
        return qml.state()
    # print(qml.matrix(circuit)())
    return circuit

def tensor_product(matrices):
    sigma_plus = np.array([[0, 1], [0, 0]])
    sigma_minus = np.array([[0, 0], [1, 0]])
    identity = np.array([[1, 0], [0, 1]])
    mlist = [sigma_plus, sigma_minus, identity]
    result = mlist[matrices[0]]
    for matrix in matrices[1:]:
        result = np.kron(result, mlist[matrix])
    return result

def get_simulate_unitary(t, bit_string):    
    # for unitary build contrary_v
    U = expm(-1j * 2 * np.pi * t * (tensor_product(bit_string)+tensor_product([not(i) for i in bit_string])))
    return U

def decompose_unitary(t, bit_string):
    unitary_gate = get_simulate_unitary(t, bit_string)
    print(unitary_gate)
    dev = qml.device('default.qubit', wires=len(bit_string))
    
    @qml.qnode(dev)
    def circuit():
        qml.QubitUnitary(unitary_gate, wires=range(len(bit_string)))
        return qml.state()
    return circuit
a = 0
if __name__ == '__main__':
    def set_print_form(suppress=True, precision=4, linewidth=300):
    # 不要截断 是否使用科学计数法 输出浮点数位数 宽度
        np.set_printoptions(threshold=np.inf, suppress=suppress, precision=precision,  linewidth=linewidth)
    set_print_form()
    import numpy as np
    def get_decompose_time(bit_string=[0, 0, 1, 1], f=None):
        t = 0.9
        time_dict = {}
        write_string = ''
        num_qubits = len(bit_string)
        time_start = perf_counter()
        qc = get_driver_component(num_qubits, t, bit_string)
        print(get_simulate_unitary(t, bit_string))
        # print(np.allclose(get_circ_unitary(qc), get_simulate_unitary(t, bit_string)))
        print(qml.draw(qc)())
        a = qml.matrix(qc)()
        print(qml.matrix(qc)())

    bit_string = [0,0,1,0,1,0,0]
    time = get_decompose_time(bit_string, None)

    # import pandas as pd
    # df = pd.DataFrame(time_list)
    # import matplotlib.pyplot as plt
    # import matplotlib as mpl
    # scale = 0.009
    # fig = plt.figure(figsize=(2500 * scale , 800 * scale))
    # mpl.rcParams['pdf.fonttype'] = 42
    # mpl.rcParams['ps.fonttype'] = 42
    # mpl.rcParams['font.family'] = 'Times New Roman'
    # mpl.rcParams['font.size'] = 62
    # mpl.rcParams['axes.unicode_minus'] = False
    # mpl.rcParams['mathtext.fontset'] = 'custom'
    # mpl.rcParams['mathtext.rm'] = 'Times New Roman'
    # mpl.rcParams['mathtext.it'] = 'Times New Roman:italic'
    # mpl.rcParams['mathtext.bf'] = 'Times New Roman:bold'
    # # set axes linewidth
    # mpl.rcParams['axes.linewidth'] = 5
    # ## set ticks linewidth
    # mpl.rcParams['xtick.major.size'] = 20
    # mpl.rcParams['xtick.major.width'] = 5
    # mpl.rcParams['ytick.major.size'] = 20
    # mpl.rcParams['ytick.major.width'] = 5
    # axes = plt.axes([0,0,1,1])
    # for i, col in enumerate(df.columns):
    #     axes.bar(df.index + 0.3 * i, df[col], width=0.3, label=col)
    # axes.legend(frameon=False, bbox_to_anchor=(1, 1), ncol=4)
    # axes.tick_params(axis='x', which='major', length=20)
    # axes.set_xlabel('# qubits')
    # axes.set_yscale('log')
    # axes.set_xticks(df.index + 0.15)
    # axes.set_xticklabels(df.index + 3)
    # axes.grid(axis='y', linestyle='--', linewidth=3, color='#B0B0B0')
    # axes.set_ylabel('Time (s)')
    # fig.savefig('implementations/figures/decompose_optimize.pdf', dpi=600, format='pdf', bbox_inches='tight')

In [ ]:
from time import perf_counter
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit_aer import Aer
from qiskit import transpile
from scipy.linalg import expm
import numpy as np


def apply_convert(qc, bit_string):
    num_qubits = len(bit_string)
    bit_string = bit_string[::-1]
    for i in range(0, num_qubits - 1):
        qc.cx(i + 1, i)
        if bit_string[i] == bit_string[i + 1]:
            qc.x(i)
    qc.h(num_qubits - 1)
    qc.x(num_qubits - 1)

def apply_reverse(qc, bit_string):
    num_qubits = len(bit_string)
    bit_string = bit_string[::-1]
    qc.x(num_qubits - 1)
    qc.h(num_qubits - 1)
    for i in range(num_qubits - 2, -1, -1):
        if bit_string[i] == bit_string[i + 1]:
            qc.x(i)
        qc.cx(i + 1, i)

def decompose_phase_gate(circuit:QuantumCircuit, qr:QuantumRegister, ancilla:QuantumRegister, phase:float) -> QuantumCircuit:
    """
    Decompose a phase gate into a series of controlled-phase gates.
    Args:
        num_phase_qubits (int): the number of qubits that the phase gate acts on.
        phase (float): the phase angle of the phase gate.
        max_num_qubit_control (int): the maximum number of qubits that the controlled-phase gates can control.
    Returns:
        QuantumCircuit: the circuit that implements the decomposed phase gate.
    """
    num_phase_qubits = len(qr)
    if num_phase_qubits == 1:
        circuit.p(phase, 0)
    elif num_phase_qubits == 2:
        circuit.cp(phase, 0, 1)
    else:
        # convert into the multi-cx gate 
        # partition qubits into two sets
        half_num_qubit = num_phase_qubits // 2
        qr1 = qr[:half_num_qubit]
        qr2 = qr[half_num_qubit:]
        circuit.rz(-phase/2, ancilla[0])
        circuit.mcx(qr1, ancilla[0], ancilla[1], mode='recursion')
        circuit.rz(phase/2, ancilla[0])
        circuit.mcx(qr2, ancilla[0], ancilla[1], mode='recursion')
        circuit.rz(-phase/2, ancilla[0])
        circuit.mcx(qr1, ancilla[0], ancilla[1], mode='recursion')
        circuit.rz(phase/2, ancilla[0])
        circuit.mcx(qr2, ancilla[0], ancilla[1], mode='recursion')

def get_driver_component(num_qubits, t, bit_string, use_decompose=True):
    qc = QuantumCircuit(num_qubits)
    apply_convert(qc, bit_string)
    qc.barrier(label='convert')
    qr = qc.qubits[:num_qubits]
    if use_decompose == True:
        ancilla = QuantumRegister(2,name='anc')
        qc.add_register(ancilla)
        decompose_phase_gate(qc, qr, ancilla, -2 * np.pi * t)
    else:
        qc.mcp(-2 * np.pi * t, list(range(1, num_qubits)), 0)
    qc.barrier(label='multi-ctrl')
    qc.x(num_qubits - 1)
    if use_decompose == True:
        decompose_phase_gate(qc, qr, ancilla, 2 * np.pi * t)
    else:
        qc.mcp(2 * np.pi * t, list(range(1, num_qubits)), 0)
    qc.x(num_qubits - 1)
    qc.barrier(label='reverse')
    apply_reverse(qc, bit_string)
    return qc

# 输入qc, 返回电路酉矩阵
def get_circ_unitary(quantum_circuit):
    backend = Aer.get_backend('unitary_simulator' )
    new_circuit = transpile(quantum_circuit, backend)
    job = backend.run(new_circuit)
    result = job.result()
    unitary = result.get_unitary()
    return unitary

def tensor_product(matrices):
    sigma_plus = np.array([[0, 1], [0, 0]])
    sigma_minus = np.array([[0, 0], [1, 0]])
    identity = np.array([[1, 0], [0, 1]])
    mlist = [sigma_plus, sigma_minus, identity]
    result = np.array(mlist[matrices[0]])
    for matrix in matrices[1:]:
        result = np.kron(result, mlist[matrix])
    return result

def get_simulate_unitary(t, bit_string):    
    # for unitary build contrary_v
    U = expm(-1j * 2 * np.pi * t * (tensor_product(bit_string)+tensor_product([not(i) for i in bit_string])))
    return U

def decompose_unitary(t, bit_string):
    unitary = get_simulate_unitary(t,bit_string)
    qc = QuantumCircuit(len(bit_string))
    qc.unitary(unitary, range(len(bit_string)))
    return qc.decompose()

def decompose_circuit(qc, max_control_qubits=5):
    from qiskit import transpile
    control_gates = []
    # do not work
    # for i in range(1, max_control_qubits + 1):
    #     control_gates.append(''.join(['c'] * i + ['z']))
    qc = transpile(qc, basis_gates=[*control_gates, 'h', 'x', 'y', 'cx', 'barrier', 'p'], optimization_level=3)
    return qc
b = 0
if __name__ == '__main__':
    def set_print_form(suppress=True, precision=4, linewidth=300):
    # 不要截断 是否使用科学计数法 输出浮点数位数 宽度
        np.set_printoptions(threshold=np.inf, suppress=suppress, precision=precision,  linewidth=linewidth)
    set_print_form()
    import numpy as np
    def get_decompose_time(bit_string=[0, 0, 1, 1], f=None):
        t = 0.9
        time_dict = {}
        write_string = ''
        num_qubits = len(bit_string)
        time_start = perf_counter()
        qc = get_driver_component(num_qubits, t, bit_string, True)
        b = get_circ_unitary(qc)
        print(b)

    time = get_decompose_time(bit_string, None)
        

In [ ]:
print(np.allclose(a, b))


In [ ]:
nonzerobits = [1,0,-1,0,1,0]
filtered_arr = [0 if x == -1 else 1 for x in nonzerobits if x != 0]
filtered_arr